In [7]:
# LOADING PACKAGES
import numpy as np
import pandas as pd
from varname import argname
import time
import keras
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score

from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Masking, Dot, Add, BatchNormalization
from keras.layers import TimeDistributed

import wandb
from wandb.keras import WandbCallback

LR, BATCH_SIZE, EPOCHS, MAX_LEN = 0.001, 128, 50, 1600
INPUT_SHAPE_1HOT, INPUT_SHAPE_2MER, INPUT_SHAPE_4MER, INPUT_SHAPE_6MER = (1600, 4), (16, 2), (256, 2), (1600, 2)

print('Packages loaded!')

# AMOUNT OF UNIQUE LABELS AT EACH TAXON LEVEL
fam_count, gen_count, spe_count = 349, 954, 1569

Packages loaded!


In [ ]:
def make_seq2tax(out_len, name = 'seq2tax'):
    input_1HOT = Input(shape = INPUT_SHAPE_1HOT)
    input_2MER = Input(shape = INPUT_SHAPE_2MER)
    input_4MER = Input(shape = INPUT_SHAPE_4MER)
    input_6MER = Input(shape = INPUT_SHAPE_6MER)


    ## CONV Layers
    X_cnn = X_mask
    # conv_net
    for i in range(CONV_NET_nr):
        X_cnn = conv_net_block(X_cnn, n_cnn_filters=NET_filters, cnn_window=NET_window)
    # res_net
    for i in range(RES_NET_nr):
        X_cnn = res_net_block(X_cnn, n_cnn_filters=NET_filters, cnn_window=NET_window)

    ## Extra Pooling layer and Dropout
    X_pool = AveragePooling1D(pool_size=POOL_s)(X_cnn)
    X_drop = Dropout(DROP_r)(X_pool)

    ## RNN Layers
    H_lstm = Bidirectional(LSTM(LSTM_nodes, return_sequences=True), merge_mode='sum')(X_drop)
    H_lstm = Activation('tanh')(H_lstm)

    ## ATT Layers
    r_emb = attention_layer(H_lstm, n_layer=ATT_layers, n_node=ATT_nodes, block_name = 'att')
        
    # Fully connected layers
    r_emb = fully_connected(r_emb, n_layer=FC_layers, n_node=FC_nodes, drop_out_rate=FC_drop, block_name = 'fc')

    # Compile model
    out_Kin = Dense(kin_count, activation='softmax', name='Kingdom_out')(r_emb)
    out_Phy = Dense(phy_count, activation='softmax', name='Phylum_out')(r_emb)
    out_Cla = Dense(cla_count, activation='softmax', name='Class_out')(r_emb)
    out_Ord = Dense(ord_count, activation='softmax', name='Order_out')(r_emb)
    out_Fam = Dense(fam_count, activation='softmax', name='Family_out')(r_emb)
    out_Gen = Dense(gen_count, activation='softmax', name='Genus_out')(r_emb)
    out_Spe = Dense(spe_count, activation='softmax', name='Species_out')(r_emb)

    R2Pmodel = Model(inputs = X, outputs = out, name = name)
    
    return R2Pmodel

In [ ]:
taxa = ['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']


In [3]:
# Read2Pheno
## Conv & Res net layers
CONV_NET_nr, RES_NET_nr, NET_filters, NET_window = 2, 1, 64, 2
## extra Dropout layer (after Res block)
DROP_r, POOL_s = 0.2, 2
## BiLSTM layer
LSTM_nodes = 128
## attention Layers
ATT_layers, ATT_nodes = 1, 128
## fully connected layers
FC_layers, FC_nodes, FC_drop = 1, 128, 0.3

# BLOCK FUNCTIONS
def conv_net_block(X, n_cnn_filters=256, cnn_window=9, block_name='convblock'):
    '''
    convolutional block with a 1D convolutional layer, a batch norm layer followed by a relu activation.
    parameters:
        n_cnn_filters: number of output channels
        cnn_window: window size of the 1D convolutional layer
    '''
    X = Conv1D(n_cnn_filters, cnn_window, strides=1, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    return X

def res_net_block(X, n_cnn_filters=256, cnn_window=9, block_name='resblock'):
    '''
    residual net block accomplished by a few convolutional blocks.
    parameters:
        n_cnn_filters: number of output channels
        cnn_window: window size of the 1D convolutional layer
    '''
    X_identity = X
    # cnn0
    X = Conv1D(n_cnn_filters, cnn_window, strides=1, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    # cnn1
    X = Conv1D(n_cnn_filters, cnn_window, strides=1, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    # cnn2
    X = Conv1D(n_cnn_filters, cnn_window, strides=1, padding='same')(X)
    X = BatchNormalization()(X)
    X = Add()([X, X_identity])
    X = Activation('relu')(X)
    return X

def attention_layer(H_lstm, n_layer, n_node, block_name='att'):
    '''
    feedforward attention layer accomplished by time distributed dense layers.
    parameters:
        n_layer: number of hidden layers
        n_node: number of hidden nodes
    '''
    H_emb = H_lstm
    for i in range(n_layer):
        H_lstm = TimeDistributed(Dense(n_node, activation="tanh"))(H_lstm)
    M = TimeDistributed(Dense(1, activation="linear"))(H_lstm)
    alpha = keras.layers.Softmax(axis=1)(M)
    r_emb = Dot(axes = 1)([alpha, H_emb])
    r_emb = Flatten()(r_emb)
    return r_emb

def fully_connected(r_emb, n_layer, n_node, drop_out_rate=0.5, block_name='fc'):
    '''
    fully_connected layer consists of a few dense layers.
    parameters:
        n_layer: number of hidden layers
        n_node: number of hidden nodes
        drop_out_rate: dropout rate to prevent the model from overfitting
    '''
    for i in range(n_layer):
        r_emb = Dense(n_node, activation="relu")(r_emb)
    r_emb = Dropout(drop_out_rate)(r_emb) 
    return r_emb

# TOTAL MODEL FUNCTION
def make_R2Pmodel(out_len, INPUT_SHAPE=INPUT_SHAPE_RNN, name='Read2Pheno'):
    X = Input(shape=INPUT_SHAPE)
    X_mask = Masking(mask_value=0.)(X)

    ## CONV Layers
    X_cnn = X_mask
    # conv_net
    for i in range(CONV_NET_nr):
        X_cnn = conv_net_block(X_cnn, n_cnn_filters=NET_filters, cnn_window=NET_window)
    # res_net
    for i in range(RES_NET_nr):
        X_cnn = res_net_block(X_cnn, n_cnn_filters=NET_filters, cnn_window=NET_window)

    ## Extra Pooling layer and Dropout
    X_pool = AveragePooling1D(pool_size=POOL_s)(X_cnn)
    X_drop = Dropout(DROP_r)(X_pool)

    ## RNN Layers
    H_lstm = Bidirectional(LSTM(LSTM_nodes, return_sequences=True), merge_mode='sum')(X_drop)
    H_lstm = Activation('tanh')(H_lstm)

    ## ATT Layers
    r_emb = attention_layer(H_lstm, n_layer=ATT_layers, n_node=ATT_nodes, block_name = 'att')
        
    # Fully connected layers
    r_emb = fully_connected(r_emb, n_layer=FC_layers, n_node=FC_nodes, drop_out_rate=FC_drop, block_name = 'fc')

    # Compile model
    out = Dense(out_len, activation='softmax', name='final_dense')(r_emb)
    R2Pmodel = Model(inputs = X, outputs = out, name = name)
    
    return R2Pmodel

In [ ]:
def train_and_evaluate_model(model, train_data, train_labels, validation_data, validation_labels, test_data, test_labels):
    wandb.init(project = 'Final Training', entity = 'bachelorprojectgroup9', name=model.name)

    print (f'Loading {model.name} model...')
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=LR), metrics=['accuracy'])
    print(model.summary())

    print (f'Fitting {model.name} model...')
    start_time = time.time()
    history = model.fit(train_data, train_labels, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data = (validation_data, validation_labels), callbacks=[WandbCallback()])
    time_taken = round(time.time() - start_time)
    # history object is saved and can later be destinguished using the model/train_data names
    np.save('history/{}_{}.npy'.format(model.name, argname('train_data')), history.history)
    
    print (f'Evaluating {model.name} model...')
    test_labels_arg = np.argmax(test_labels, axis=1)
    test_predictions = np.argmax(model.predict(test_data), axis=1)
    loss, accuracy = model.evaluate(test_data, test_labels)

    # F1-score: harmonic mean of the precision and recall
    #   score from 0 to 1
    f1 = f1_score(y_true=test_labels_arg, y_pred=test_predictions, average='weighted')
    # Matthews correlation coefficient: coefficient of +1 represents a perfect prediction,
    #   0 an average random prediction and -1 an inverse prediction
    mcc = matthews_corrcoef(y_true=test_labels_arg, y_pred=test_predictions)

    score_dict = pd.DataFrame({'Model/run' : model.name, 'Data' : argname('train_data'), 'Training time' : time_taken, 'Test loss' : loss, 'Test accuracy' : accuracy, 'F1-score' : f1, 'MCC' : mcc}, index=[0])
    print(score_dict)
    # score metrics are saved and can later be destinguished using the model names
    score_dict.to_csv(f'scores/{model.name}_evaluation.csv', index=False)

    wandb.finish()
    return